In [1]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

import gradio as gr
import pandas as pd

from virny.datasets import (ACSIncomeDataset, GermanCreditDataset, LawSchoolDataset, 
                            CardiovascularDiseaseDataset, BankMarketingDataset, DiabetesDataset2019)
from virny.custom_classes.metrics_interactive_visualizer import MetricsInteractiveVisualizer
from source.custom_classes.database_client import DatabaseClient
from configs.constants import (EXP_COLLECTION_NAME, GERMAN_CREDIT_DATASET, BANK_MARKETING_DATASET, CARDIOVASCULAR_DISEASE_DATASET,
                               DIABETES_DATASET, LAW_SCHOOL_DATASET, ACS_INCOME_DATASET)

In [2]:
def read_metrics_from_db(dataset_names: list, null_imputers: list, db_collection_name: str):
    db = DatabaseClient()
    db.connect()
    
    dataset_metrics_dct = dict()
    for dataset_name in dataset_names:
        dataset_metrics_dct[dataset_name] = dict()
        for imputer_idx, null_imputer in enumerate(null_imputers):
            # Extract experimental data for the defined dataset from MongoDB
            # query = {'dataset_name': dataset_name, 'null_imputer_name': null_imputer,  'tag': 'demo_20240423'}
            query = {'dataset_name': dataset_name, 'null_imputer_name': null_imputer,  'tag': 'OK'}
            records = db.execute_read_query(db_collection_name, query)
            model_metric_df = pd.DataFrame(records)

            # Capitalize column names to be consistent across the whole library
            new_column_names = []
            for col in model_metric_df.columns:
                new_col_name = '_'.join([c.capitalize() for c in col.split('_')])
                new_column_names.append(new_col_name)

            model_metric_df.columns = new_column_names
            model_metric_df = model_metric_df.drop(columns=['Model_Params', 'Tag', 'Model_Init_Seed', 'Runtime_In_Mins'])
            model_metric_df['Model_Name'] = (model_metric_df['Model_Name'] + '__' + 
                                             model_metric_df['Null_Imputer_Name'] + '_' + 
                                             model_metric_df['Virny_Random_State'].astype(str))
            if imputer_idx == 0:
                dataset_metrics_dct[dataset_name][null_imputer] = model_metric_df
            else:
                dataset_metrics_dct[dataset_name][null_imputer] = (
                    pd.concat([dataset_metrics_dct[dataset_name][null_imputer], model_metric_df], axis=0))
                dataset_metrics_dct[dataset_name][null_imputer] = dataset_metrics_dct[dataset_name][null_imputer].reset_index(drop=True)

            print(f'Extracted metrics for {dataset_name} dataset and {null_imputer} imputer')

    db.close()

    return dataset_metrics_dct

In [3]:
# Define configs for sample datasets
demo_configs = {
    ACS_INCOME_DATASET: {
        'data_loader': ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                                        subsample_size=15_000, subsample_seed=42),
        'sensitive_attributes_dct': {'SEX': '2', 'RAC1P': ['2', '3', '4', '5', '6', '7', '8', '9'], 'SEX&RAC1P': None},
    },
    LAW_SCHOOL_DATASET: {
        'data_loader': LawSchoolDataset(),
        'sensitive_attributes_dct': {'male': '0', 'race': 'Non-White', 'male&race': None},
    },
    GERMAN_CREDIT_DATASET: {
        'data_loader': GermanCreditDataset(),
        'sensitive_attributes_dct': {'sex': 'female', 'age': [19, 20, 21, 22, 23, 24, 25], 'sex&age': None},
    },
    CARDIOVASCULAR_DISEASE_DATASET: {
        'data_loader': CardiovascularDiseaseDataset(),
        'sensitive_attributes_dct': {'gender': '1'},
    },
    BANK_MARKETING_DATASET: {
        'data_loader': BankMarketingDataset(),
        'sensitive_attributes_dct': {'age': [18, 19, 20, 21, 22, 23, 24, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75,
                                             76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 95]} # gap between 89 and 95
    },
    DIABETES_DATASET: {
        'data_loader': DiabetesDataset2019(),
        'sensitive_attributes_dct': {'Gender': 'Female'},
    },
}

In [4]:
dataset_metrics_dct = read_metrics_from_db(dataset_names=list(demo_configs.keys()),
                                           null_imputers=['baseline'],
                                           db_collection_name=EXP_COLLECTION_NAME)

Extracted metrics for folk dataset and baseline imputer
Extracted metrics for law_school dataset and baseline imputer
Extracted metrics for german dataset and baseline imputer
Extracted metrics for heart dataset and baseline imputer
Extracted metrics for bank dataset and baseline imputer
Extracted metrics for diabetes dataset and baseline imputer


In [5]:
for dataset_name in dataset_metrics_dct.keys():
    model_metric_df = dataset_metrics_dct[dataset_name]['baseline']
    
    # Create columns based on values in the Subgroup column
    pivoted_model_metric_df = model_metric_df.pivot(columns='Subgroup', values='Metric_Value',
                                                     index=[col for col in model_metric_df.columns
                                                            if col not in ('Subgroup', 'Metric_Value')]).reset_index()
    pivoted_model_metric_df = pivoted_model_metric_df.rename_axis(None, axis=1)

    dataset_metrics_dct[dataset_name]['baseline'] = pivoted_model_metric_df

In [6]:
# Create gradio demo objects for each sample dataset
dataset_names = list(demo_configs.keys())
sample_demos = []
for dataset_name in dataset_names:
    sample_demo = MetricsInteractiveVisualizer(
        X_data=demo_configs[dataset_name]['data_loader'].X_data,
        y_data=demo_configs[dataset_name]['data_loader'].y_data,
        model_metrics=dataset_metrics_dct[dataset_name]['baseline'],
        sensitive_attributes_dct=demo_configs[dataset_name]['sensitive_attributes_dct']
    ).create_web_app(start_app=False)
    sample_demos.append(sample_demo)

IMPORTANT: You are using gradio version 4.10.0, however version 4.29.0 is available, please upgrade.
--------


In [7]:
# Build a web application with tabs for each sample dataset
demo = gr.TabbedInterface(sample_demos, [name.replace('_', ' ') for name in dataset_names], theme=gr.themes.Soft())
demo.launch(inline=False, debug=True, show_error=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
IMPORTANT: You are using gradio version 4.10.0, however version 4.29.0 is available, please upgrade.
--------


Traceback (most recent call last):
  File "/Users/denys_herasymuk/Research/NYU/ML_Lifecycle_Project/Code/data-cleaning-stability/vldb_venv/lib/python3.9/site-packages/gradio/queueing.py", line 459, in call_prediction
    output = await route_utils.call_process_api(
  File "/Users/denys_herasymuk/Research/NYU/ML_Lifecycle_Project/Code/data-cleaning-stability/vldb_venv/lib/python3.9/site-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/denys_herasymuk/Research/NYU/ML_Lifecycle_Project/Code/data-cleaning-stability/vldb_venv/lib/python3.9/site-packages/gradio/blocks.py", line 1533, in process_api
    result = await self.call_function(
  File "/Users/denys_herasymuk/Research/NYU/ML_Lifecycle_Project/Code/data-cleaning-stability/vldb_venv/lib/python3.9/site-packages/gradio/blocks.py", line 1151, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/denys_herasymuk/Research/NYU/ML_Lifecycl

Keyboard interruption in main thread... closing server.
